In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import requests
import json
import requests
import time
from datetime import datetime
from IPython.display import display, clear_output
import numpy as np
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
target_diff = 0.005
stop_loss = 0.005
units = 5000
profit = (target_diff / 0.0001)*(units/10000)
loss = (stop_loss / 0.0001)*(units/10000)
version = '10'

In [4]:
filename_date = '100919'

layout = pd.read_csv('../data/' + filename_date + 'mds_out_layout.csv')
layout = layout['features'].values
data = pd.read_csv('../data/' + filename_date + 'mds_out.csv', usecols=layout)
data.shape

(16415, 785)

In [5]:
data['time'].min()

'2017-01-19 21:00:00'

In [6]:
data['time'].max()

'2019-09-24 19:00:00'

In [7]:
num_cols = ['ask_c', 'bid_c']

In [8]:
data['time'] = pd.to_datetime(data['time'])
data = data.sort_values(by='time')
data = data.drop_duplicates(subset=['time'])
data = data.reset_index(drop=True)
data['index'] = data.index
for n in num_cols:
    data[n] = pd.to_numeric(data[n])
data['target_tp'] = data['ask_c'] + target_diff
data['target_sl'] = data['bid_c'] - stop_loss
data = data[~data['target_tp'].isnull()]

In [9]:
data['diff'] = data['target_tp'] - data['target_sl']
data['diff'].describe()

count    16415.000000
mean         0.010155
std          0.000097
min          0.010090
25%          0.010120
50%          0.010130
75%          0.010140
max          0.011500
Name: diff, dtype: float64

In [10]:
# data[['index', 'time', 'volume', 'bid_o', 'bid_h','bid_l','bid_c', 'ask_o', 'ask_h','ask_l','ask_c', 'ask_prev_c', 'bid_prev_c', 'target_tp', 'target_sl', 'g', 'l', 'l_time']].head(40)

In [11]:
data['g']=np.NaN
data['l']=np.NaN
deep=len(data.index)
irange = data['index'].values
tp_good = False
sl_good = False

for i in irange:

    while tp_good == False and sl_good == False:
        show = (i / len(irange))*100
        show = str(round(show, 2))
        show = show + ' - ' + str(datetime.now())
        display(show)
        tp_good = False
        sl_good = False
    #     print(i)

        for r in irange:
    #             print(i, r, tp_good, sl_good)
            if r > i:
                if tp_good == False and sl_good == False:
                    if  data[data['index'] == r]['ask_h'].values[0] >= data[data['index'] == i]['target_tp'].values[0] and not tp_good:
                        data.at[i, 'g'] = data[data['index'] == r]['index'].values[0]
                        data.at[i, 'g_time'] = data[data['index'] == r]['time'].values[0]
                        tp_good = True
                    else:
                        data.at[i, 'g'] = None
                        data.at[i, 'g_time'] =None

                    if  data[data['index'] == r]['bid_l'].values[0] <= data[data['index'] == i]['target_sl'].values[0] and not sl_good:
                        data.at[i, 'l'] = data[data['index'] == r]['index'].values[0]
                        data.at[i, 'l_time'] = data[data['index'] == r]['time'].values[0]
                        sl_good = True
                    else:  
                        data.at[i, 'l'] = None     
                        data.at[i, 'l_time'] = None  
                else:
                    break

        clear_output()    

#     print(datetime.now())

In [12]:
# START WITH FIRST TARGET INDEX BEC TIME ISSUE

if tp_good:
    target_index = data['g'].max()
    good_data = data[~data['g'].isnull()].copy()

if sl_good:
    target_index = data['l'].max()
    good_data = data[~data['g'].isnull()].copy()

tp_good = False
sl_good = False
done = False

counter = 1
while not done:
    show = (target_index / len(irange))*100
    show = str(round(show, 2))
    show = show + ' - ' + str(datetime.now())
    display(show)
    tp_good = False
    sl_good = False

    for r in irange:
        if r > target_index:
            if tp_good == False and sl_good == False:
                if  data[data['index'] == r]['ask_h'].values[0] >= data[data['index'] == target_index]['target_tp'].values[0] and not tp_good:
                    data.at[target_index, 'g'] = data[data['index'] == r]['index'].values[0]
                    data.at[target_index, 'g_time'] = data[data['index'] == r]['time'].values[0]
                    tp_good = True
                else:
                    data.at[target_index, 'g'] = None
                    data.at[target_index, 'g_time'] =None


                if  data[data['index'] == r]['bid_l'].values[0] <= data[data['index'] == target_index]['target_sl'].values[0] and not sl_good:
                    data.at[target_index, 'l'] = data[data['index'] == r]['index'].values[0]
                    data.at[target_index, 'l_time'] = data[data['index'] == r]['time'].values[0]
                    sl_good = True

                else:  
                    data.at[i, 'l'] = None     
                    data.at[i, 'l_time'] = None  
            else:
                break
    
    if tp_good == True:
        target_index = data['g'].max()
        try:
            temp = data[data['g'] == target_index].copy()
            temp2 = data[data['index'] == target_index].copy()

            if counter == 1:
                good_data = temp
                counter += 1
            else:
                good_data = good_data.append(temp)

            if target_index == data['index'].max():
                done = True
        except IndexError:
            done = True

    elif sl_good == True:
        target_index = data['l'].max()
        try:
            temp = data[data['l'] == target_index].copy()
            temp2 = data[data['index'] == target_index].copy()

            if counter == 1:
                good_data = temp
                counter += 1
            else:
                good_data = good_data.append(temp)

            if target_index == data['index'].max():
                done = True
        except IndexError:
            done = True
            
    else:
        done = True


    clear_output()    

In [13]:
# good_mask = (data['g'].isnull()) & (data['l'].isnull())
# good_data = data[~good_mask].copy()
good_data.shape

(588, 793)

In [14]:
good_data[['time', 'volume', 'bid_o', 'bid_h','bid_l','bid_c', 'ask_o', 'ask_h','ask_l','ask_c', 'target_tp', 'target_sl', 'g', 'l', 'l_time']].head(10)

,time,volume,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,target_tp,target_sl,g,l,l_time
19,2017-01-20 16:00:00,5856,1.06806,1.07087,1.06797,1.07043,1.06820,1.07103,1.06811,1.07062,1.07562,1.06543,45.0,NaN,NaN
45,2017-01-23 18:00:00,1123,1.07427,1.07682,1.07423,1.07621,1.07443,1.07707,1.07436,1.07657,1.08157,1.07121,NaN,79.0,2017-01-25 04:00:00
79,2017-01-25 04:00:00,7874,1.07208,1.07265,1.07106,1.07129,1.07221,1.07278,1.07119,1.07143,1.07643,1.06629,84.0,NaN,NaT
84,2017-01-25 09:00:00,6850,1.07519,1.07693,1.07442,1.07451,1.07533,1.07706,1.07458,1.07462,1.07962,1.06951,NaN,109.0,2017-01-26 10:00:00
109,2017-01-26 10:00:00,11127,1.07103,1.07120,1.06880,1.06964,1.07116,1.07133,1.06893,1.06977,1.07477,1.06464,NaN,156.0,2017-01-30 09:00:00
156,2017-01-30 09:00:00,11299,1.06607,1.06650,1.06256,1.06286,1.06620,1.06661,1.06270,1.06303,1.06803,1.05786,159.0,NaN,NaT
159,2017-01-30 12:00:00,8437,1.06583,1.06921,1.06527,1.06871,1.06602,1.06933,1.06538,1.06883,1.07383,1.06371,179.0,NaN,NaT
179,2017-01-31 08:00:00,11088,1.07007,1.07517,1.06986,1.07501,1.07020,1.07533,1.06998,1.07513,1.08013,1.07001,183.0,NaN,NaT
183,2017-01-31 12:00:00,12255,1.07894,1.08101,1.07787,1.08093,1.07924,1.08115,1.07803,1.08106,1.08606,1.07593,NaN,206.0,2017-02-01 11:00:00
206,2017-02-01 11:00:00,11967,1.07597,1.07624,1.07497,1.07570,1.07609,1.07637,1.07511,1.07592,1.08092,1.07070,224.0,NaN,NaT


In [15]:
def get_target(value):
    if value > 0:
        return 1
    else:
        return 0

In [16]:
# max_date = data['time'].max()
good_data['g_time'] = pd.to_datetime(good_data['g_time'])
good_data['l_time'] = pd.to_datetime(good_data['l_time'])

good_data['g'] = pd.to_numeric(good_data['g'])
good_data['l'] = pd.to_numeric(good_data['l'])

good_data['g'] = good_data['g'].fillna(0)
good_data['l'] = good_data['l'].fillna(0)

# data['g'] = data['g'].fillna(data['index'].max())
# data['l'] = data['l'].fillna(data['index'].max())

good_data['long_target'] = good_data['g'].apply(get_target)
good_data['short_target'] = good_data['l'].apply(get_target)

print(good_data['long_target'].value_counts())
print()
print(good_data['short_target'].value_counts())

1    296
0    292
Name: long_target, dtype: int64

1    294
0    294
Name: short_target, dtype: int64


In [17]:
# good_data['g_time_shift'] = good_data['g_time'].shift(1)
# good_data['g_time_shift'] = pd.to_datetime(good_data['g_time_shift'] )
good_data['time'] = pd.to_datetime(good_data['time'] )
# good_data['g_diff'] = good_data['g_time'] - good_data['g_time_shift']
# good_data['g_diff'] = good_data['g_diff'].dt.seconds

In [18]:
# good_data['order'] = good_data.index
# t1 = good_data[good_data['g'] != 0.0].drop_duplicates(subset='g')
# t2 = good_data[good_data['l'] != 0.0].drop_duplicates(subset='l')
# good_data = pd.concat([t1, t2], axis=0)
# good_data= good_data.sort_values(by='order')
good_data['same_time'] = good_data['g_time'] == good_data['l_time']
good_data['same_time'] = good_data['same_time'].apply(lambda x: 1 if x else 0)
good_data = good_data[good_data['same_time'] == 0]
good_data.shape

(586, 796)

In [19]:
pos = good_data[good_data['long_target'] == 1].copy()
print(pos.shape)
neg = good_data[good_data['long_target'] == 0].copy()

pos['time_diff'] = (pos['g_time'] - pos['time']) / np.timedelta64(1, 'h')
pos_bad = pos[pos['time_diff'] > 24].copy()
pos_bad['long_target'] = 0

pos = pos[pos['time_diff'] <= 24]

pos.drop(['time_diff'], axis=1, inplace=True)
pos_bad.drop(['time_diff'], axis=1, inplace=True)

good_data = pd.concat([pos, neg, pos_bad], axis=0)
good_data.shape

(294, 796)


(586, 796)

In [20]:
'../data/' + filename_date + '_mds_out_final_v' + version + '_long_live.csv'

'../data/100919_mds_out_final_v10_long_live.csv'

In [21]:
good_data.to_csv('../data/' + filename_date + '_mds_out_final_v' + version + '_long_live.csv', index=False, header=True)

In [22]:
good_data[['time', 'bid_c', 'ask_c','target_tp', 'target_sl', 'g', 'l', 'g_time', 'l_time', 'long_target', 'short_target', 'time']].head(10)

,time,bid_c,ask_c,target_tp,target_sl,g,l,g_time,l_time,long_target,short_target,time
79,2017-01-25 04:00:00,1.07129,1.07143,1.07643,1.06629,84.0,0.0,2017-01-25 09:00:00,NaT,1,0,2017-01-25 04:00:00
156,2017-01-30 09:00:00,1.06286,1.06303,1.06803,1.05786,159.0,0.0,2017-01-30 12:00:00,NaT,1,0,2017-01-30 09:00:00
159,2017-01-30 12:00:00,1.06871,1.06883,1.07383,1.06371,179.0,0.0,2017-01-31 08:00:00,NaT,1,0,2017-01-30 12:00:00
179,2017-01-31 08:00:00,1.07501,1.07513,1.08013,1.07001,183.0,0.0,2017-01-31 12:00:00,NaT,1,0,2017-01-31 08:00:00
206,2017-02-01 11:00:00,1.07570,1.07592,1.08092,1.07070,224.0,0.0,2017-02-02 05:00:00,NaT,1,0,2017-02-01 11:00:00
456,2017-02-15 21:00:00,1.06205,1.06218,1.06718,1.05705,471.0,0.0,2017-02-16 12:00:00,NaT,1,0,2017-02-15 21:00:00
559,2017-02-22 04:00:00,1.05077,1.05089,1.05589,1.04577,568.0,0.0,2017-02-22 13:00:00,NaT,1,0,2017-02-22 04:00:00
648,2017-02-27 21:00:00,1.05787,1.05801,1.06301,1.05287,663.0,0.0,2017-02-28 12:00:00,NaT,1,0,2017-02-27 21:00:00
715,2017-03-02 16:00:00,1.05010,1.05021,1.05521,1.04510,732.0,0.0,2017-03-03 09:00:00,NaT,1,0,2017-03-02 16:00:00
732,2017-03-03 09:00:00,1.05567,1.05578,1.06078,1.05067,739.0,0.0,2017-03-03 16:00:00,NaT,1,0,2017-03-03 09:00:00


In [23]:
shorts = good_data[good_data['short_target'] == 1].copy()
shorts['time'] = pd.to_datetime(shorts['time'])
shorts['l_time'] = pd.to_datetime(shorts['l_time'] )
shorts['time_diff'] = shorts['l_time'] - shorts['time']
shorts['time_diff'].head() 

45    1 days 10:00:00
84    1 days 01:00:00
109   3 days 23:00:00
183   0 days 23:00:00
224   0 days 11:00:00
Name: time_diff, dtype: timedelta64[ns]

In [24]:
shorts['time_diff'] = shorts['time_diff']  / np.timedelta64(1, 'h')

In [25]:
shorts['time_diff'].describe()

count    292.000000
mean      39.821918
std       40.836727
min        1.000000
25%       11.000000
50%       23.000000
75%       54.750000
max      202.000000
Name: time_diff, dtype: float64

In [26]:
dcounts = good_data['time'].value_counts().reset_index()
dcounts.columns = ['time', 'count']

In [27]:
24*5

120